In [13]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "XzTt9hf8DAsmOuUa" #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

## TODO, place bet only using these three arguments
myRaceVenue = 'sunderland-bags'
myRaceID = '1.200362898'
liability_amount = 1500

In [14]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)
runners_df



,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,29.0,7.84,38.0,9.90,29.0,39.81,ACTIVE,None,None
1,41419586,16.5,5.71,19.5,13.25,17.0,61.96,ACTIVE,None,None
2,41419587,17.5,6.21,23.0,9.60,18.5,28.14,ACTIVE,None,None
3,41419588,55.0,23.17,90.0,17.05,NaN,0.00,ACTIVE,None,None
4,41419589,23.0,10.39,27.0,51.63,23.0,28.80,ACTIVE,None,None
5,41419592,55.0,6.40,75.0,5.34,55.0,13.12,ACTIVE,None,None
6,41419593,50.0,11.28,95.0,14.40,60.0,7.56,ACTIVE,None,None
7,41419594,55.0,18.36,85.0,18.45,50.0,0.52,ACTIVE,None,None
8,41419595,150.0,7.41,280.0,12.63,1.2,84.23,ACTIVE,None,None
9,41419596,65.0,10.87,90.0,5.63,55.0,17.70,ACTIVE,None,None


In [15]:
#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)

print(lay_options_ids)

[41419585, 41419586, 41419587, 41419588, 41419589, 41419592, 41419593, 41419594, 41419595, 41419596, 41419599, 41419600, 41419601, 41419602, 41419603, 41419606, 41419607, 41419608, 41419609, 41419610, 41419613, 41419614, 41419615, 41419616, 41419617, 41419620, 41419621, 41419622, 41419623, 41419624]


In [16]:
from utils import choose_lay_option

lay_selection_index = choose_lay_option(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    # assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print(lay_selection_id)
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

limit_order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=liability_amount # use this if i want $1500 as liability
)

instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

['4.00', '10.00', '4.60', '3.70', '15.00', '2.80']
['2.80', '3.70', '4.00', '4.60', '10.00', '15.00']
6,4
28
Lay option found successfully
28
41419623


{'orderType': 'LIMIT',
 'selectionId': '41419623',
 'side': 'LAY',
 'limitOrder': {'price': 15.0,
  'persistenceType': 'LAPSE',
  'betTargetType': 'PAYOUT',
  'betTargetSize': 1500}}

In [17]:
# order = trading.betting.place_orders(
#     market_id = myRaceID,
#     customer_strategy_ref='Naing_maker',
#     instructions=[instructions_filter]
# )

In [19]:
# # listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
#                                                     market_ids=[myRaceID])

# # 
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['clearedOrders'])

,eventTypeId,eventId,marketId,selectionId,handicap,betId,placedDate,persistenceType,orderType,side,betOutcome,priceRequested,settledDate,lastMatchedDate,betCount,priceMatched,priceReduced,sizeSettled,profit,customerStrategyRef
0,4339,31536095,1.200287438,41419623,0.0,271878680306,2022-06-19T14:27:57.000Z,LAPSE,LIMIT,LAY,WON,15.0,2022-06-19T14:31:58.000Z,2022-06-19T14:27:57.000Z,1,15.0,False,100.0,100.0,Naing_maker
1,4339,31536095,1.200287438,41419623,0.0,271878717259,2022-06-19T14:28:15.000Z,LAPSE,LIMIT,LAY,WON,15.5,2022-06-19T14:31:58.000Z,2022-06-19T14:28:15.000Z,1,15.5,False,5.0,5.0,NaN
